In [1]:
# import basic packages
import os
import numpy as np
import wget
import sys
import gdown
import zipfile
import librosa
import pandas as pd
# in the notebook, we only can use one GPU

In [2]:
def create_path(path):
    if not os.path.exists(path):
        os.mkdir(path)

workspace = "./workspace"
dataset_path = os.path.join(workspace, "esc-50")
esc_raw_path = os.path.join(dataset_path, 'raw')

create_path(workspace)
create_path(dataset_path)

In [3]:
if not os.path.exists(os.path.join(dataset_path, 'ESC-50-master.zip')):
    print("-------------Downloading ESC-50 Dataset-------------")
    wget.download('https://github.com/karoldvl/ESC-50/archive/master.zip', out=dataset_path)
    with zipfile.ZipFile(os.path.join(dataset_path, 'ESC-50-master.zip'), 'r') as zip_ref:
        zip_ref.extractall(esc_raw_path)
    print("-------------Success-------------")

-------------Downloading ESC-50 Dataset-------------
-------------Success-------------


In [4]:
meta_path = os.path.join(esc_raw_path, 'ESC-50-master', 'meta', 'esc50.csv')
audio_path = os.path.join(esc_raw_path, 'ESC-50-master', 'audio')
resample_path = os.path.join(dataset_path, 'resample')
savedata_path = os.path.join(dataset_path, 'esc-50-data.npy')
create_path(resample_path)

meta = np.loadtxt(meta_path , delimiter=',', dtype='str', skiprows=1)
audio_list = os.listdir(audio_path)

In [8]:
# resample
print("-------------Resample ESC-50-------------")
for f in audio_list:
    full_f = os.path.join(audio_path, f)
    resample_f = os.path.join(resample_path, f)
    if not os.path.exists(resample_f):
        os.system('sox -V1 ' + full_f + ' -r 32000 ' + resample_f)
print("-------------Success-------------")

print("-------------Build Dataset-------------")
output_dict = [[] for _ in range(5)]
for label in meta:
    name = label[0]
    fold = label[1]
    target = label[2]
    y, sr = librosa.load(os.path.join(resample_path, name), sr = None)
    output_dict[int(fold) - 1].append(
        {
            "name": name,
            "target": int(target),
            "waveform": y
        }
    )
np.save(savedata_path, output_dict)
print("-------------Success-------------")

-------------Resample ESC-50-------------
-------------Success-------------
-------------Build Dataset-------------
-------------Success-------------


In [13]:
import pandas as pd
df = pd.read_csv('./workspace/esc-50/raw/ESC-50-master/meta/esc50.csv')
df.head()

,filename,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,1,0,dog,True,100032,A
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A


In [2]:
from dataloader.dataset import ESC_Dataset

In [3]:
dataset = ESC_Dataset(meta_data='./workspace/esc-50/raw/ESC-50-master/meta/esc50.csv', audio_path='./workspace/esc-50/resample' , fold=[1, 2, 3, 4])

fold: [1, 2, 3, 4]
dataset length: 1600


In [4]:
dataset.meta_data

,filename,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,1,0,dog,True,100032,A
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A
...,...,...,...,...,...,...,...
1595,4-99193-B-4.wav,4,4,frog,False,99193,B
1596,4-99644-A-4.wav,4,4,frog,False,99644,A
1597,4-99644-B-4.wav,4,4,frog,False,99644,B
1598,4-99644-C-4.wav,4,4,frog,False,99644,C


In [5]:
x, y = dataset[0]

In [6]:
x.shape[1] / 32000

5.0

In [7]:
y.shape

()